In [1]:
import numpy as np
import pylab as plt
import pandas as pd
import os
import segyio as sio
from src import segyrw
from src import dsp
import ipywidgets as ipw
import tensorflow as tf
from tqdm.notebook import tqdm
from scipy import signal

/tmp/ipykernel_15960/306516062.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2025-01-19 15:37:51.412274: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-19 15:37:51.506145: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has al

In [2]:
import CNNCreator

In [18]:
model = CNNCreator.model_conv1d(
    3,
    shape=1000,
    depth=4,
    filters=32,
    kernel_size=32,
    channels=1,
    activation='relu',
    last_activation='softmax',
    last_kernel_size=32,
    dropout=.5,
    batch_norm=True,
    acivation_after_batch=False,
    pooling=None,
    lr=0.001,)
model.load_weights('../fb_picking_notebooks/models/model_weights_mount')

In [19]:
data = np.load("../FBdata/npy/with_picks/Kluchi2014.npy")

In [20]:
result = model.predict(data)

26/26 [==============================] - 0s 12ms/step


In [27]:
result.shape

(808, 1000, 3)

In [21]:
picks = np.argmax(result[:,:,1], axis=1)

In [22]:
picks

array([946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946,
       946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946,
       946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946,
       946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946,
       946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946,
       946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946,
       946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946,
       946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946,
       946, 946, 946,  55, 946, 946, 946,  45,  74,  80, 946, 946, 946,
       946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946,
       946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946,
       946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946,
       946, 946, 946, 946, 946, 946, 946,  57, 946, 946, 946, 946, 946,
       946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 94

In [23]:
true_picks = np.load("../FBdata/npy/with_picks/Kluchi2014_picks.npy")

In [24]:
true_picks

array([  0,  37,  61,  82,  92, 115, 136, 147, 151, 153, 156, 161, 165,
       163, 159, 156, 158, 156, 157, 155, 155, 155, 154, 150, 148, 151,
       149, 148, 147, 146, 147, 149,  36,   0,  31,  62,  75,  80,  90,
       100, 116, 121, 138, 141, 155, 157, 156, 156, 151, 150, 151, 148,
       148, 150, 147, 146, 147, 145, 145, 142, 141, 140, 138, 140, 142,
       141, 138, 139, 141, 139, 141,  67,  61,  32,   0,  33,  51, 106,
       126, 146, 148, 146, 146, 146, 144, 142, 139, 138, 140, 136, 136,
       135, 136, 135, 134,  96,  92,  85,  80,  74,  68,  62,  56,  50,
        43,  38,  32,  27,  21,   0,  19,  26,  33,  38,  43,  48,  54,
        59,  75,  97, 120, 125, 131, 130, 131, 131, 130, 129, 129, 131,
       128, 126, 128, 125, 124, 124, 125, 124, 123, 123, 123, 123, 125,
       123, 126, 128, 126, 127, 125, 125, 120, 115, 107,  97,  90,  88,
        76,  67,  58,  44,  39,  33,   0,  31,  54,  60,  65,  82,  87,
       103, 106, 110, 115, 120, 122, 123, 122, 121, 121, 123, 12

In [25]:
diff_picks = abs(picks - true_picks)

In [26]:
diff_picks

array([946, 909, 885, 864, 854, 831, 810, 799, 795, 793, 790, 785, 781,
       783, 787, 790, 788, 790, 789, 791, 791, 791, 792, 796, 798, 795,
       797, 798, 799, 800, 799, 797, 910, 946, 915, 884, 871, 866, 856,
       846, 830, 825, 808, 805, 791, 789, 790, 790, 795, 796, 795, 798,
       798, 796, 799, 800, 799, 801, 801, 804, 805, 806, 808, 806, 804,
       805, 808, 807, 805, 807, 805, 879, 885, 914, 946, 913, 895, 840,
       820, 800, 798, 800, 800, 800, 802, 804, 807, 808, 806, 810, 810,
       811, 810, 811, 812, 850, 854, 861, 866, 872, 878, 884, 890, 896,
       903, 908, 914,  28, 925, 946, 927,  19,  41,  42, 903, 898, 892,
       887, 871, 849, 826, 821, 815, 816, 815, 815, 816, 817, 817, 815,
       818, 820, 818, 821, 822, 822, 821, 822, 823, 823, 823, 823, 821,
       823, 820, 818, 820, 819, 821, 821, 826, 831, 839, 849, 856, 858,
       870, 879, 888, 902, 907, 913, 946,  26, 892, 886, 881, 864, 859,
       843, 840, 836, 831, 826, 824, 823, 824, 825, 825, 823, 82